Khai báo thư viện

In [2]:
import pandas as pd

In [3]:
# Đọc file data.pkl
df = pd.read_pickle('data/clean/data.pkl')
df.head()

,Tên hãng xe,Tên dòng xe,Năm sản xuất,Giá,Tỉnh,Tình trạng,Xuất xứ,Kiểu dáng,Hộp số,Động cơ,Màu ngoại thất,Màu nội thất,Số chỗ ngồi,Số cửa,Dẫn động,Số Km đã đi
0,Suzuki,XL7 1.5 AT,2023.0,530000000.0,TP HCM,Xe mới,Nhập khẩu,SUV,Số tự động,Xăng 1.5 L,Vàng,Đen,7.0,5.0,FWD - Dẫn động cầu trước,NaN
1,Suzuki,Super Carry Truck 1.0 MT,2022.0,249000000.0,Hà Nội,Xe mới,Lắp ráp trong nước,Truck,Số tay,Xăng 1.0 L,Trắng,Ghi,2.0,2.0,RFD - Dẫn động cầu sau,NaN
2,Suzuki,Super Carry Van Blind Van,2022.0,215000000.0,Hà Nội,Xe đã dùng,Lắp ráp trong nước,Van/Minivan,Số tay,Xăng 1.0 L,Trắng,Ghi,2.0,5.0,RFD - Dẫn động cầu sau,11000.0
3,Suzuki,Jimny 1.5L 4x4 AT,2023.0,790000000.0,Hà Nội,Xe đã dùng,Nhập khẩu,SUV,Số tự động,Xăng 1.5 L,Xám,Xám,4.0,3.0,4WD - Dẫn động 4 bánh,0.0
4,Suzuki,Swift GLX 1.2 AT,2020.0,450000000.0,Hà Nội,Xe đã dùng,Nhập khẩu,Hatchback,Số tự động,Xăng 1.2 L,Đỏ,Đen,5.0,5.0,FWD - Dẫn động cầu trước,30000.0


Tiền xử lý trước khi huấn luyện

In [4]:
# Ánh xạ các cột phân loại
dan_dong = 

SyntaxError: invalid syntax (1140625704.py, line 2)